In [1]:
import os
os.chdir('/Users/bnowacki/Documents/Git Repositories/rapid-soh-estimation-from-short-pulses')

from rapid_soh_estimation.rapid_soh_estimation.config import *
from rapid_soh_estimation.rapid_soh_estimation.common_methods import *


cc_data = load_processed_data(data_type='cc')
pulse_data = load_processed_data(data_type='slowpulse')


# get modeling data (charge pulse only, 20soc only)
all_data = deepcopy(pulse_data)
idxs = np.where((all_data['pulse_type'] == 'chg') & (all_data['soc'] == 20))
for k in all_data.keys():
	all_data[k] = all_data[k][idxs]
all_data['voltage_rel'] = np.asarray([v - v[0] for v in all_data['voltage']])
modeling_data = create_modeling_data(all_data=all_data, input_feature_keys=['voltage_rel'])


In [13]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
import optuna

# Procedure:
#   1. select a model
#   2. define a set of hyperparameters
#   3. optimize hyperparameters
#   4. use optimal hyperparamters / create optimal model
#   5. fit model and run CV
#   6a. For linear models: average model coeffs over CV and print
#	6b. For RF model: average the feature importance values (Gini importance)


def loss_function(y_true, y_pred):
	"""The loss function used for all optuna studies

	Args:
		y_true (MatrixLike | ArrayLike): Ground truth (correct) target values
		y_pred (MatrixLike | ArrayLike): Estimated target values

	Returns:
		_type_: The loss for this prediction (ie, prediction error)
	"""

	loss = mean_squared_error(y_true, y_pred)
	return loss

class OptunaHyperParamOptimization:
	def __init__(self, X, y, splits, loss_fnc, random_state):
		self.X = X
		self.y = y
		self.splits = splits
		self.loss_fnc = loss_fnc
		self.random_state = random_state

	def __call__(self, trial:optuna.trial.Trial):
		return None

class OptunaHyperParamOptimization_Ridge(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggest alpha value from Optuna search space
			alpha = trial.suggest_float('alpha', 0.0, 1e9)
			# create Ridge model
			model = Ridge(
				alpha=alpha,
				random_state=self.random_state
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_Lasso(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			alpha = trial.suggest_float('alpha', 0, 1e9)

			# create Lasso model
			model = Lasso(
				alpha=alpha,
				random_state=self.random_state
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_ElasticNet(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			alpha = trial.suggest_float('alpha', 0.0, 1e9)
			l1_ratio = trial.suggest_float('l1_ratio', 0.01, 1.0)

			# create ElasticNet model
			model = ElasticNet(
				alpha=alpha, 
				l1_ratio=l1_ratio, 
				random_state=self.random_state,
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)

class OptunaHyperParamOptimization_RandomForestRegressor(OptunaHyperParamOptimization):
	def __call__(self, trial:optuna.trial.Trial):
		# average the loss over all cross-validation splits
		total_loss = 0
		for train_idxs, test_idxs in self.splits:
			# get suggested model parameters from Optuna search space
			n_estimators = trial.suggest_int('n_estimators', 5, 500)
			min_samples_split = trial.suggest_float('min_samples_split', 0.01, 1.0)
			min_samples_leaf = trial.suggest_float('min_samples_leaf', 0.01, 1.0)
			max_features = trial.suggest_float('max_features', 0.01, 1.0)
			max_samples = trial.suggest_float('max_samples', 0.01, 1.0)

			# create RandomForestRegressor model
			model = RandomForestRegressor(
				n_estimators=n_estimators, 
				min_samples_split=min_samples_split,
				min_samples_leaf=min_samples_leaf,
				max_features=max_features,  
				max_samples=max_samples,
				n_jobs=-1,
			)

			# standardize input and output data (using only the training data to create the scaler)
			scaler_X = StandardScaler().fit(self.X[train_idxs])
			scaler_y = StandardScaler().fit(self.y[train_idxs])
			X_sc = scaler_X.transform(self.X)
			y_sc = scaler_y.transform(self.y)

			# fit model to scaled input and output data
			model.fit(X_sc[train_idxs], y_sc[train_idxs])
			
			# get predictions
			y_pred_sc = model.predict(X_sc[test_idxs])

			# add loss to total
			total_loss += self.loss_fnc(y_sc[test_idxs], y_pred_sc)

		# return average cross-validation loss
		return total_loss / len(self.splits)



random_state = 13

cv_splitter = Custom_CVSplitter(n_splits=3, split_type='group_id', rand_seed=random_state)
cv_splits = list(cv_splitter.split(
	X = modeling_data['model_input'], 
	y = modeling_data['model_output'], 
	cell_ids = modeling_data['cell_id']))

dir_hyperparam_results = dir_repo_main.joinpath("results", 'hyperparameter_optimization')
dir_hyperparam_results.mkdir(exist_ok=True, parents=True)

# key = study_name
hyperparam_opt_params = {
	'Hyperparameters_Ridge': {
		'objective':
			OptunaHyperParamOptimization_Ridge(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':100,
		'save_results':True,
		'filename':'hyperparam_study_ridge'
	},
	'Hyperparameters_Lasso': {
		'objective':
			OptunaHyperParamOptimization_Lasso(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':5,
		'save_results':True,
		'filename':'hyperparam_study_lasso'
	},
	'Hyperparameters_ElasticNet': {
		'objective':
			OptunaHyperParamOptimization_ElasticNet(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':5,
		'save_results':True,
		'filename':'hyperparam_study_elasticnet'
	},
	'Hyperparameters_RandomForestRegressor': {
		'objective':
			OptunaHyperParamOptimization_RandomForestRegressor(
				X=modeling_data['model_input'], 
				y=modeling_data['model_output'], 
				splits=cv_splits, 
				loss_fnc=loss_function, 
				random_state=random_state
			),
		'n_trials':5,
		'save_results':True,
		'filename':'hyperparam_study_randomforest'
	},
}


def print_optuna_study_results(study:optuna.Study):
	"""Prints the best loss and parameters for a given Optuna study"""
	
	print()
	print('*'*100)
	print(f'  Study: {study.study_name}')
	print('*'*100)
	print('  Best Loss: ', study.best_trial.value)
	print('  Best Params: ')
	for k,v in study.best_trial.params.items():
		print(f'    {k}: {v}')
	print('*'*100)
	print()

def perform_hyperparam_optimization(params:dict):
	for key,val in params.items():
		study = optuna.create_study(
			study_name=key,
			direction='minimize', 
			sampler=optuna.samplers.TPESampler(seed=random_state),)
		study.optimize(
			func = val['objective'],
			n_trials = val['n_trials'],
			n_jobs=-1)
		if val['save_results']:
			pickle.dump(
				study, 
				open(dir_hyperparam_results.joinpath(f"{val['filename']}.pkl"), 'wb'),
				protocol=pickle.HIGHEST_PROTOCOL)
		else:
			print_optuna_study_results(study)



perform_hyperparam_optimization(hyperparam_opt_params)

# print best hyperparameters for all studies in dir_hyperparam_results
for file_res in dir_hyperparam_results.glob('*.pkl'):
	study = pickle.load(open(file_res, 'rb'))
	print_optuna_study_results(study)



[I 2024-09-25 12:04:53,062] A new study created in memory with name: Hyperparameters_Lasso
[I 2024-09-25 12:04:53,202] Trial 1 finished with value: 1.005039425745166 and parameters: {'alpha': 629343464.2368525}. Best is trial 1 with value: 1.005039425745166.
[I 2024-09-25 12:04:53,240] Trial 4 finished with value: 1.005039425745166 and parameters: {'alpha': 409453534.79587364}. Best is trial 1 with value: 1.005039425745166.
[I 2024-09-25 12:04:53,323] Trial 0 finished with value: 1.005039425745166 and parameters: {'alpha': 748793361.7200595}. Best is trial 1 with value: 1.005039425745166.
[I 2024-09-25 12:04:53,335] Trial 3 finished with value: 1.005039425745166 and parameters: {'alpha': 424257570.36071867}. Best is trial 1 with value: 1.005039425745166.
[I 2024-09-25 12:04:53,344] Trial 2 finished with value: 1.005039425745166 and parameters: {'alpha': 796918475.2766232}. Best is trial 1 with value: 1.005039425745166.
[I 2024-09-25 12:04:53,345] A new study created in memory with name


****************************************************************************************************
  Study: Hyperparameters_RandomForestRegressor
****************************************************************************************************
  Best Loss:  1.0045919427665917
  Best Params: 
    n_estimators: 40
    min_samples_split: 0.5313796586609383
    min_samples_leaf: 0.3508255757943977
    max_features: 0.22316970660747334
    max_samples: 0.3030590628662242
****************************************************************************************************


****************************************************************************************************
  Study: Hyperparameters_Lasso
****************************************************************************************************
  Best Loss:  1.005039425745166
  Best Params: 
    alpha: 629343464.2368525
****************************************************************************************************


*******